In [1]:
# Imports, dims

import cadquery as cq
from IPython.display import clear_output, display

import thread

mm = 1
inch = 25.4 * mm


class Dims:
    cilinder_radius = 5.9 / 2
    cube_offset = 1.2
    cube_height = 4.4
    cillinder_height = 5.3

    thread_len = 20.0
    rod_radius = 6.0
    rod_height = 50.0

In [2]:
def make_adapter_head() -> cq.Workplane:
    cube = (
        cq.Workplane("XY")
        .moveTo(-Dims.cilinder_radius, 0)
        .lineTo(-Dims.cilinder_radius, Dims.cilinder_radius + Dims.cube_offset)
        .lineTo(Dims.cilinder_radius, Dims.cilinder_radius + Dims.cube_offset)
        .lineTo(Dims.cilinder_radius, 0)
        .lineTo(-Dims.cilinder_radius, 0)
        .close()
        .extrude(Dims.cube_height)
    )
    cillinder = cq.Workplane("XY").circle(Dims.cilinder_radius).extrude(Dims.cillinder_height)
    return cube.union(cillinder)


# clear_output(wait=True)
# display(make_adapter_head())

In [3]:
def make_sampler() -> cq.Workplane:
    sampler_height = 5
    sample_base = cq.Workplane("XY").rect(20, 20).extrude(sampler_height)

    return sample_base.union(make_adapter_head().translate((0, 0, sampler_height)))


clear_output(wait=True)
display(make_sampler())
sampler = make_sampler()
sampler.export("sampler.stl")
pass

In [4]:
def make_rod() -> cq.Workplane:
    rod_thread = thread.IsoThread(
        major_diameter=0.25 * inch,  # ¼ inch = 6.35mm
        pitch=(1 / 20) * inch,  # 20 TPI = 1.27mm pitch
        length=Dims.thread_len,  # Adjust length as needed
        external=True,  # External/male thread
        end_finishes=("square", "square"),  # or ("fade", "square") for tapered ends
        hand="right",  # Standard right-hand thread
    )

    rod_core = cq.Workplane("XY").circle(rod_thread.min_radius).extrude(Dims.thread_len * 1.1)

    complete_thread = cq.Workplane().add(rod_thread).union(rod_core.translate((0, 0, -1.1)))

    adapter = make_adapter_head()

    rod = cq.Workplane("XY").circle(Dims.rod_radius).extrude(5).translate((0, 0, Dims.thread_len))

    all = cq.Workplane().union(complete_thread).union(rod)

    return all


rod = make_rod()
clear_output(wait=True)
rod.export("rod_sample.stl")
display(rod)
